In [1]:
def Modularity(array,cluster):  #有向网络的模块度
    #总边数
    m = np.sum(array) #对整个01矩阵求和，是有向网络中的总边数
    k1 = np.sum(array,axis=1) #axis指第二个维度，对第二个维度进行求和   求行和，每个节点的出度，返回一维数组
    k2 = np.sum(array,axis=0) #axis指第一个维度，对第一个维度进行求和   求列和，每个节点的入度，返回一维数组
    k2 = k2.reshape(k2.shape[0],1)
    #节点度数的乘积
    k1k2 = (k1*k2).T
    #任意两点连接边数的期望值
    Eij = k1k2/m
    #节点v和w的实际边数与随机网络下边数期望之差
    B = array - Eij
    #获取节点、社区矩阵
    node_cluster = np.dot(cluster,np.transpose(cluster))  #将cluster矩阵转置
    results = np.dot(B,node_cluster)
    #求和
    sum_results = np.trace(results) #这里求迹是因为上一步原矩阵与社区矩阵相乘之后，所求的即迹
    #模块度计算
    Q = sum_results / m
    return Q
def Get_cluster(G,pi):
    cluster=np.zeros((len(G),len(pi)),dtype='int32') # 'int32'数组中的每个元素是32位的整数
    #len(G)图中节点的数量，即图的大小
    #cluster是节点数×社区数的全整数0矩阵*
    s=0
    for i in pi:           #每个社区                              
        #j为点的标号
        for j in list(i):                     
            #将同一个社区的点对应位置赋值为1
            cluster[j][s]=1
        #一个社区赋值完成，进行下一个社区，直到全部赋值完成
        s=s+1
    return cluster


#coding=utf-8
# 首先导入包
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
class GN(object):
    
    """docstring for GN"""
    def __init__(self, G):
        self._G_cloned = G.copy()
        self._G = G
        self._partition = [[n for n in G.nodes()]]
        self._max_Q = 0.0

    # GN算法
    def execute(self):
        while len( self._G_cloned.edges()) > 0:
            # 1.计算所有边的edge betweenness
            edge = max(nx.edge_betweenness( self._G_cloned).items(), 
                key = lambda item:item[1])[0]
            # 2.移去edge betweenness最大的边
            self._G_cloned.remove_edge(edge[0], edge[1])
            # 获得移去边后的子连通图
            components = [list(c) for c in list(nx.strongly_connected_components( self._G_cloned))]
            if len(components) != len(self._partition):
                # 3.计算Q值
                cur_Q = Modularity(s,Get_cluster(self._G,components))
                if cur_Q > self._max_Q:
                    self._max_Q = cur_Q
                    self._partition = components
        return self._partition,self._max_Q

if __name__ == '__main__':
    # 加载网络并可视化
    import time
    start=time.time()
    data=pd.read_csv(r"E:\jupyter notebook program\matrix_convote.csv",header=None)
    #data=pd.read_csv(r"E:\jupyter notebook program\matrix_emailEucore.csv",header=None)
    #data=pd.read_csv(r"E:\jupyter notebook program\matrix_moreno_innovation.csv",header=None)
    #data=pd.read_csv(r"E:\jupyter notebook program\matrix_celegans_n306.csv",header=None)
    #data=pd.read_csv(r"E:\jupyter notebook program\matrix_Friendship.csv",header=None)
    #ata=pd.read_csv(r"E:\jupyter notebook program\matrix_wiki_talk.csv",header=None)
        
    #data=pd.read_csv(r"D:\看论文\14.小型LFR\LFR-1\matrix_data500边.csv",header=None)
    #data=pd.read_csv(r"D:\看论文\14.小型LFR\LFR-2\matrix_data500边.csv",header=None)
    #data=pd.read_csv(r"D:\看论文\14.小型LFR\LFR-3\matrix_data500边.csv",header=None)
    #data=pd.read_csv(r"D:\看论文\14.小型LFR\LFR-4\matrix_data500边.csv",header=None)
    #data=pd.read_csv(r"D:\看论文\14.小型LFR\LFR-5\matrix_data500边.csv",header=None)
    #data=pd.read_csv(r"D:\看论文\14.小型LFR\LFR-6\matrix_data500边.csv",header=None)
    #data=pd.read_csv(r"D:\看论文\14.小型LFR\LFR-7\matrix_data500边.csv",header=None)
    data=pd.read_csv(r"D:\1.看论文0\14.小型LFR\LFR-8\matrix_data500边.csv",header=None)
    
    s=np.array(data)
    G = nx.DiGraph(s) #有向

    # GN算法
    algo = GN(G)
    partition = algo.execute()
    print("模块度",partition[1])
    end=time.time()
    t=end-start
    print("时间：",t)

模块度 0.32340800000000003
时间： 22.345252990722656
